In [ ]:
pip install pyspark

     |████████████████████████████████| 212.3MB 67kB/s 
     |████████████████████████████████| 204kB 38.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=acb7c37e724c1feec35489871d4c435021e7768a5c71e003891abfe31b643d01
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ML project").getOrCreate()

In [ ]:
df = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/iris.data",header=True,inferSchema=True)
df.printSchema()

root
 |-- 5.1: double (nullable = true)
 |-- 3.5: double (nullable = true)
 |-- 1.4: double (nullable = true)
 |-- 0.2: double (nullable = true)
 |-- Iris-setosa: string (nullable = true)



In [ ]:
k=len(df.columns)
for i in range(k):
  old_name = df.columns[i]
  new_name = "f"+str(i)
  print(new_name)
  df=df.withColumnRenamed(old_name,new_name)

f0
f1
f2
f3
f4


In [ ]:
df.show()

In [ ]:
import pandas as pd
pd.DataFrame(df.take(5),columns=df.columns).transpose()

In [ ]:
class_name = df.columns[len(df.columns)-1]
df.groupBy(class_name).count().toPandas()
print(class_name)

f4


In [ ]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer()
indexer.setInputCol(class_name).setOutputCol("label")
df1=indexer.fit(df).transform(df)

In [ ]:
df1.show()

+---+---+---+---+-----------+-----+
| f0| f1| f2| f3|         f4|label|
+---+---+---+---+-----------+-----+
|4.9|3.0|1.4|0.2|Iris-setosa|  2.0|
|4.7|3.2|1.3|0.2|Iris-setosa|  2.0|
|4.6|3.1|1.5|0.2|Iris-setosa|  2.0|
|5.0|3.6|1.4|0.2|Iris-setosa|  2.0|
|5.4|3.9|1.7|0.4|Iris-setosa|  2.0|
|4.6|3.4|1.4|0.3|Iris-setosa|  2.0|
|5.0|3.4|1.5|0.2|Iris-setosa|  2.0|
|4.4|2.9|1.4|0.2|Iris-setosa|  2.0|
|4.9|3.1|1.5|0.1|Iris-setosa|  2.0|
|5.4|3.7|1.5|0.2|Iris-setosa|  2.0|
|4.8|3.4|1.6|0.2|Iris-setosa|  2.0|
|4.8|3.0|1.4|0.1|Iris-setosa|  2.0|
|4.3|3.0|1.1|0.1|Iris-setosa|  2.0|
|5.8|4.0|1.2|0.2|Iris-setosa|  2.0|
|5.7|4.4|1.5|0.4|Iris-setosa|  2.0|
|5.4|3.9|1.3|0.4|Iris-setosa|  2.0|
|5.1|3.5|1.4|0.3|Iris-setosa|  2.0|
|5.7|3.8|1.7|0.3|Iris-setosa|  2.0|
|5.1|3.8|1.5|0.3|Iris-setosa|  2.0|
|5.4|3.4|1.7|0.2|Iris-setosa|  2.0|
+---+---+---+---+-----------+-----+
only showing top 20 rows



In [ ]:
(training_data,test_data)=df1.randomSplit([0.8,0.2])

In [ ]:
training_data.toPandas()

,f0,f1,f2,f3,f4,label
0,4.3,3.0,1.1,0.1,Iris-setosa,2.0
1,4.4,2.9,1.4,0.2,Iris-setosa,2.0
2,4.4,3.0,1.3,0.2,Iris-setosa,2.0
3,4.4,3.2,1.3,0.2,Iris-setosa,2.0
4,4.5,2.3,1.3,0.3,Iris-setosa,2.0
...,...,...,...,...,...,...
111,7.4,2.8,6.1,1.9,Iris-virginica,1.0
112,7.7,2.8,6.7,2.0,Iris-virginica,1.0
113,7.7,3.0,6.1,2.3,Iris-virginica,1.0
114,7.7,3.8,6.7,2.2,Iris-virginica,1.0


In [ ]:
class_name="label"
feature_name=df.columns[:-1]
print(class_name)
print(feature_name)


label
['f0', 'f1', 'f2', 'f3']


In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler()
assembler.setInputCols(feature_name).setOutputCol("features")
transformed_data = assembler.transform(df1)

In [ ]:
transformed_data.toPandas()

,f0,f1,f2,f3,f4,label,features
0,4.9,3.0,1.4,0.2,Iris-setosa,2.0,"[4.9, 3.0, 1.4, 0.2]"
1,4.7,3.2,1.3,0.2,Iris-setosa,2.0,"[4.7, 3.2, 1.3, 0.2]"
2,4.6,3.1,1.5,0.2,Iris-setosa,2.0,"[4.6, 3.1, 1.5, 0.2]"
3,5.0,3.6,1.4,0.2,Iris-setosa,2.0,"[5.0, 3.6, 1.4, 0.2]"
4,5.4,3.9,1.7,0.4,Iris-setosa,2.0,"[5.4, 3.9, 1.7, 0.4]"
...,...,...,...,...,...,...,...
144,6.7,3.0,5.2,2.3,Iris-virginica,1.0,"[6.7, 3.0, 5.2, 2.3]"
145,6.3,2.5,5.0,1.9,Iris-virginica,1.0,"[6.3, 2.5, 5.0, 1.9]"
146,6.5,3.0,5.2,2.0,Iris-virginica,1.0,"[6.5, 3.0, 5.2, 2.0]"
147,6.2,3.4,5.4,2.3,Iris-virginica,1.0,"[6.2, 3.4, 5.4, 2.3]"


In [ ]:
(training_data, test_data) = transformed_data.randomSplit([0.8,0.2])

In [ ]:
from pyspark.ml.classification import LogisticRegression
model = LogisticRegression(featuresCol = 'features',labelCol=class_name, maxIter=30)

In [ ]:
M = model.fit(training_data)

In [ ]:
predictions = M.transform(test_data)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'label', metricName = 'accuracy')
print('Logistic Regression Accuracy:', multi_evaluator.evaluate(predictions))

Logistic Regression Accuracy: 0.9696969696969697
